In [1]:
import pysam

In [2]:
bam = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/data/03_mapping/pantrx_general_mapping/genomic/7_NI2_GM18489.bam'


In [3]:
pysam.__file__

'/Users/fairliereese/miniconda3/envs/pysam/lib/python3.13/site-packages/pysam/__init__.py'

In [4]:
afile = pysam.AlignmentFile(bam, 'rb')

In [7]:
import array 

def count_coverage(afile,
                   contig,
                   start=None,
                   stop=None,
                   region=None,
                   quality_threshold=15,
                   read_callback='all',
                   reference=None,
                   end=None):
    """count the coverage of genomic positions by reads in :term:`region`.

    The region is specified by :term:`contig`, `start` and `stop`.
    :term:`reference` and `end` are also accepted for backward
    compatibility as synonyms for :term:`contig` and `stop`,
    respectively.  Alternatively, a `samtools`_ :term:`region`
    string can be supplied.  The coverage is computed per-base [ACGT].

    Parameters
    ----------

    contig : string
        reference_name of the genomic region (chromosome)

    start : int
        start of the genomic region (0-based inclusive). If not
        given, count from the start of the chromosome.

    stop : int
        end of the genomic region (0-based exclusive). If not given,
        count to the end of the chromosome.

    region : string
        a region string.

    quality_threshold : int
        quality_threshold is the minimum quality score (in phred) a
        base has to reach to be counted.

    read_callback: string or function

        select a call-back to ignore reads when counting. It can
        be either a string with the following values:

        ``all``
            skip reads in which any of the following
            flags are set: BAM_FUNMAP, BAM_FSECONDARY, BAM_FQCFAIL,
            BAM_FDUP

        ``nofilter``
            uses every single read

        Alternatively, `read_callback` can be a function
        ``check_read(read)`` that should return True only for
        those reads that shall be included in the counting.

    reference : string
        backward compatible synonym for `contig`

    end : int
        backward compatible synonym for `stop`

    Raises
    ------

    ValueError
        if the genomic coordinates are out of range or invalid.

    Returns
    -------

    four array.arrays of the same length in order A C G T : tuple

    """

    # cdef uint32_t contig_length = afile.get_reference_length(contig)
    # cdef int _start = start if start is not None else 0
    # cdef int _stop = stop if stop is not None else contig_length
    # _stop = _stop if _stop < contig_length else contig_length

    contig_length = afile.get_reference_length(contig)
    _start = start if start is not None else 0
    _stop = stop if stop is not None else contig_length
    _stop = _stop if _stop < contig_length else contig_length

    if _stop == _start:
        raise ValueError("interval of size 0")
    if _stop < _start:
        raise ValueError("interval of size less than 0")

    # cdef int length = _stop - _start
    # cdef c_array.array int_array_template = array.array('L', [])
    # cdef c_array.array count_a
    # cdef c_array.array count_c
    # cdef c_array.array count_g
    # cdef c_array.array count_t
    # count_a = c_array.clone(int_array_template, length, zero=True)
    # count_c = c_array.clone(int_array_template, length, zero=True)
    # count_g = c_array.clone(int_array_template, length, zero=True)
    # count_t = c_array.clone(int_array_template, length, zero=True)

    length = _stop - _start
    int_array_template = ['L', 0]
    count_a = [int_array_template for _ in range(length)]
    count_c = [int_array_template for _ in range(length)]
    count_g = [int_array_template for _ in range(length)]
    count_t = [int_array_template for _ in range(length)]

    # # Data structures to store read names
    # cdef list read_names_a, read_names_c, read_names_g, read_names_t
    # read_names_a = [[] for _ in range(length)]
    # read_names_c = [[] for _ in range(length)]
    # read_names_g = [[] for _ in range(length)]
    # read_names_t = [[] for _ in range(length)]

    read_names_a = [[] for _ in range(length)]
    read_names_c = [[] for _ in range(length)]
    read_names_g = [[] for _ in range(length)]
    read_names_t = [[] for _ in range(length)]

    # cdef AlignedSegment read
    # cdef cython.str seq
    # cdef c_array.array quality
    # cdef int qpos
    # cdef int refpos
    # cdef int c = 0
    # cdef int filter_method = 0

    c = 0
    filter_method = 0
    
    if read_callback == "all":
        filter_method = 1
    elif read_callback == "nofilter":
        filter_method = 2

    # cdef int _threshold = quality_threshold or 0
    _threshold = quality_threshold or 0
    
    for read in afile.fetch(contig=contig,
                           reference=reference,
                           start=start,
                           stop=stop,
                           end=end,
                           region=region):
        # apply filter
        if filter_method == 1:
            # filter = "all"
            if (read.flag & (0x4 | 0x100 | 0x200 | 0x400)):
                continue
        elif filter_method == 2:
            # filter = "nofilter"
            pass
        else:
            if not read_callback(read):
                continue

        # count
        seq = read.seq
        if seq is None:
            continue
        quality = read.query_qualities

        for qpos, refpos in read.get_aligned_pairs(True):
            if qpos is not None and refpos is not None and \
               _start <= refpos < _stop:

                # only check base quality if _threshold > 0
                if (_threshold and quality and quality[qpos] >= _threshold) or not _threshold:
                    if seq[qpos] == 'A':
                        count_a[refpos - _start][1] += 1
                        read_names_a[refpos - _start].append(read.query_name)
                    if seq[qpos] == 'C':
                        count_c[refpos - _start][1] += 1
                        read_names_c[refpos - _start].append(read.query_name)
                    if seq[qpos] == 'G':
                        count_g[refpos - _start][1] += 1
                        read_names_g[refpos - _start].append(read.query_name)
                    if seq[qpos] == 'T':
                        count_t[refpos - _start][1] += 1
                        read_names_t[refpos - _start].append(read.query_name)

    return count_a, count_c, count_g, count_t, \
      read_names_a, read_names_c, read_names_g, read_names_t

In [8]:
# fwd strand, does pretty good
variant = 'chr12:6,534,825'
sample = 'YRI2'
# ACGT
# A : 187 (1%, 134+, 53- )
# C : 19435 (53%, 8406+, 11029- )
# G : 17213 (47%, 7513+, 9700- )
# T : 120 (0%, 46+, 74- )
count_coverage(afile, 'chr12', start=6534825-1, stop=6534825)

([['L', 31189]],
 [['L', 31189]],
 [['L', 31189]],
 [['L', 31189]],
 [['06ed8f17-b4ca-4e62-a519-5168f2393d64:0',
   '13406719-c02f-4a59-b821-5f4f6452c382:0',
   'a1c66b53-21e1-47c4-9e01-b75743736e4e:0',
   'cbe9aacb-b448-4925-ac11-fe7b46bf0a3d:0',
   'ee5d3373-f9ea-4cba-ad6b-e5059f7d5fe9;752d356a-f661-424d-9aa4-a6f5a7f5e855:1',
   '8ed8030f-88ed-4c8d-bc9b-c65712928051:0',
   '763e6b98-d50c-4545-8fc8-dc2d7a3d32f3:0',
   '8d0c787f-fc0f-4e9f-a694-088921e0c410:0',
   '38a72dbb-de82-4e49-9a38-43538e466d52:0',
   '45885a98-3977-42d8-8f87-78f0d4d0727b:0',
   '1348ec8e-e75f-4367-9137-f1452f94e6bb:0',
   '9ea1988a-4a68-4cdc-8a94-07c7f2a2343b:0',
   '3627440d-133e-4354-8f8a-131213d8b55d:0',
   'ed2aa825-ef28-4f33-89e5-a8e95273cb85:0',
   'c8e6a7bf-0de5-4425-917e-a82b11df19e4:0',
   '187aeff4-0278-4ff5-90fc-2009782c44c0:0',
   '1bf70666-44ee-420b-a37b-0938622c4f59:0',
   '4e760fd0-d514-4a38-9414-c74201c077b1:0',
   'e596b9e2-e2d1-40fa-b9ca-5248860f39ee:0',
   '682aea24-068e-4a7f-bc81-ca71c5c525d5

In [16]:
# rev strand
variant = 'chr12:6,748,870'
sample = 'YRI2'
bam = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/data/03_mapping/pantrx_general_mapping/genomic/7_NI2_GM18489.bam'
afile.count_coverage('chr12', start=6748870-1, stop=6748870)
# AGCT
# profile from IGV
# Total count: 597
# A : 0
# C : 290 (49%, 155+, 135- )
# G : 1 (0%, 1+, 0- )
# T : 306 (51%, 171+, 135- )
# N : 0
# DEL: 12
# INS: 5

(array('L', [0]), array('L', [217]), array('L', [0]), array('L', [239]))

In [17]:
# rev strand from lorals
variant = 'chr6_31268790_T_C'
sample = 'YRI2'
bam = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/data/03_mapping/pantrx_general_mapping/genomic/7_NI2_GM18489.bam'
afile.count_coverage('chr6', start=31268790-1, stop=31268790)
# AGCT
# profile from IGV
# Total count: 5753
# A : 27 (0%, 14+, 13- )
# C : 3945 (69%, 2258+, 1687- )
# G : 18 (0%, 6+, 12- )
# T : 1763 (31%, 1008+, 755- )
# N : 0
# DEL: 88
# INS: 13

(array('L', [1]), array('L', [3437]), array('L', [2]), array('L', [1496]))

In [ ]:
# ok so start = pos-1, stop=pos works